<a href="https://colab.research.google.com/github/japnit01/Face-detection-and-recognition/blob/main/Face_Recoginition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from keras.models import load_model
# from inception_model import model
import matplotlib.pyplot as plt
import os
import numpy as np
import random
from keras.layers import Flatten,Dense,Input,concatenate,Conv2D
from keras.models import Model
from keras.activations import sigmoid

In [3]:
# from sklearn.decomposition import PCA

# class PCAPlotter(tf.keras.callbacks.Callback):
    
#     def __init__(self, plt, embedding_model, x_test, y_test):
#         super(PCAPlotter, self).__init__()
#         self.embedding_model = embedding_model
#         self.x_test = x_test
#         self.y_test = y_test
#         self.fig = plt.figure(figsize=(9, 4))
#         self.ax1 = plt.subplot(1, 2, 1)
#         self.ax2 = plt.subplot(1, 2, 2)
#         plt.ion()
        
#         self.losses = []
    
#     def plot(self, epoch=None, plot_loss=False):
#         x_test_embeddings = self.embedding_model.predict(self.x_test)
#         pca_out = PCA(n_components=2).fit_transform(x_test_embeddings)
#         self.ax1.clear()
#         self.ax1.scatter(pca_out[:, 0], pca_out[:, 1], c=self.y_test, cmap='seismic')
#         if plot_loss:
#             self.ax2.clear()
#             self.ax2.plot(range(epoch), self.losses)
#             self.ax2.set_xlabel('Epochs')
#             self.ax2.set_ylabel('Loss')
#         self.fig.canvas.draw()
    
#     def on_train_begin(self, logs=None):
#         self.losses = []
#         self.fig.show()
#         self.fig.canvas.draw()
#         self.plot()
        
#     def on_epoch_end(self, epoch, logs=None):
#         self.losses.append(logs.get('loss'))
#         self.plot(epoch+1, plot_loss=True)

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(x_train.shape)

(60000, 28, 28)


In [5]:
x_train = x_train/255
x_test = x_test/255

In [6]:
x_train = np.expand_dims(x_train,axis=3)
x_test = np.expand_dims(x_train,axis=3)
print(x_train.shape)

(60000, 28, 28, 1)


In [7]:
def create_batch(batch_size=256):
	x_anchors = np.zeros((batch_size,28,28,1))
	x_positives = np.zeros((batch_size,28,28,1))
	x_negatives = np.zeros((batch_size,28,28,1))

	for i in range(0, batch_size):
        
		random_index = random.randint(0, x_train.shape[0] - 1)
		x_anchor = x_train[random_index]
		y = y_train[random_index]

		indices_for_pos = np.squeeze(np.where(y_train == y))
		indices_for_neg = np.squeeze(np.where(y_train != y))

		x_positive = x_train[indices_for_pos[random.randint(0, len(indices_for_pos) - 1)]]
		x_negative = x_train[indices_for_neg[random.randint(0, len(indices_for_neg) - 1)]]

		x_anchors[i] = x_anchor
		x_positives[i] = x_positive
		x_negatives[i] = x_negative

	return [x_anchors, x_positives, x_negatives]


In [8]:
emb_size = 64

m =tf.keras.applications.InceptionV3(
        include_top=False, weights='imagenet', input_tensor=None,
        input_shape=(96,96,3), pooling=None, classes=10,
        classifier_activation='softmax'
    )

m.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 47, 47, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 47, 47, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 47, 47, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [9]:
i = Input(shape=(28,28,1))
r = tf.keras.layers.experimental.preprocessing.Resizing(96,96)(i)
print(r.shape)
conv2d = Conv2D(3,(1,1),activation='relu')(r)
m = m(conv2d)

# flatten = m
flatten = Flatten()(m)
embmodel = Dense(64, activation="relu")(flatten)
embmodel = Dense(emb_size, activation='sigmoid')(embmodel)

embmodel = Model(inputs=i, outputs=embmodel)
embmodel.summary()

(None, 96, 96, 1)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
resizing (Resizing)          (None, 96, 96, 1)         0         
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 96, 96, 3)         6         
_________________________________________________________________
inception_v3 (Functional)    (None, 1, 1, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                131136    
_________________________________________________________________
dense_1 (Dense)              (None, 64)    

In [10]:
input_anchor = Input(shape=(28,28,1))
input_positive = Input(shape=(28,28,1))
input_negative = Input(shape=(28,28,1))

embedding_anchor = embmodel(input_anchor)
embedding_positive = embmodel(input_positive)
embedding_negative = embmodel(input_negative)

output = concatenate([embedding_anchor, embedding_positive, embedding_negative], axis=1)

net = Model([input_anchor, input_positive, input_negative], output)
net.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 64)           21938086    input_3[0][0]                    
                                                                 input_4[0][0]              

In [11]:
alpha = 0.2

def triplet_loss(y_true, y_pred):
    anchor, positive, negative = y_pred[:,:emb_size], y_pred[:,emb_size:2*emb_size], y_pred[:,2*emb_size:]
    positive_dist = tf.reduce_mean(tf.square(anchor - positive), axis=1)
    negative_dist = tf.reduce_mean(tf.square(anchor - negative), axis=1)
    return tf.maximum(positive_dist - negative_dist + alpha, 0.)

In [12]:
def data_generator(batch_size=32):
    while True:
        x = create_batch(batch_size)
        y = np.zeros((batch_size, 3*emb_size))
        yield x, y

In [13]:
net.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])

data_path = r'/content/drive/MyDrive/Colab Notebooks/data'
save_path = os.path.join(data_path, "model_weights_triplet_loss_2048.h5")

In [14]:
batch_size = 32
epochs = 50
steps_per_epoch = 100
model_history = net.fit(
  data_generator(batch_size),
  epochs=epochs, steps_per_epoch=steps_per_epoch,
  verbose=1
)

Epoch 1/50
100/100 [==============================] - 858s 8s/step - loss: 0.1748 - accuracy: 0.0279
Epoch 2/50
100/100 [==============================] - 841s 8s/step - loss: 0.1688 - accuracy: 0.0734
Epoch 3/50
100/100 [==============================] - 845s 8s/step - loss: 0.1592 - accuracy: 0.0435
Epoch 4/50
100/100 [==============================] - 838s 8s/step - loss: 0.1646 - accuracy: 0.0387
Epoch 5/50
100/100 [==============================] - 839s 8s/step - loss: 0.1667 - accuracy: 0.0345
Epoch 6/50
100/100 [==============================] - 841s 8s/step - loss: 0.1488 - accuracy: 0.0404
Epoch 7/50
 24/100 [======>.......................] - ETA: 10:39 - loss: 0.1600 - accuracy: 0.0492

KeyboardInterrupt: ignored

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
net.save_weights(save_path)